<h1 align="center"> TRABALHO - REST API - GRUPO 3</h1>

<h2> INTRODUÇÃO </h2>

<p> O objetivo do trabalho é de filtrar notícias disponíveis por uma API de acordo com um tema, salvando os resultados em uma base de dados/arquivo, tratar esses dados para a geração de relatórios e disponibilizar a consulta dos produtos gerados através de uma API utilizando o Flask. </p>
<p>A base deve ser atualizada com requisições realizadas a cada hora na fonte dos artigos: https://newsapi.org/ </p>

<h2> Metodologia </h2>

<p> O trabalho proposto estruturou sua execução da seguinte forma:

1. um script ("requisicao.py") para chamada da API 'newsapi' com os parâmetros de estudo, construindo a base de dados com arquivo .csv e atualizando
o mesmo sempre que um novo artigo for encontrado, periodicamente;

2. um script ("selects.py") para a criação dos 'relatórios', que serão consultados na API de consulta ao público;

3. um script ("flask_api.py") para manter ativa uma REST API, com os endpoints de consulta dos dados gerais, dados pelo ID e relatórios;

</p>

<p> O trecho abaixo corresponde ao script de consulta da API e criação/atualização da base de dados. Foram criadas duas funções principais para isso.</p>


In [ ]:
import pandas as pd
import requests
from datetime import datetime


apiKey = "df97b6ec3d3148c2a0f34e552e3f53af"
keyWords = "bitcoin+blockchain+web3"
today = datetime.now().strftime('%Y-%m-%d')

def busca_tema():

    url = f'https://newsapi.org/v2/everything?q={keyWords}&from=2024-03-10&to={today}&language=en&sortBy=publishedAt&page=1&apiKey={apiKey}'
    response = requests.get(url)

    if response.status_code == 200:

      results = response.json()
      print("Requisição feita com sucesso!.")
    else:
      return print("Problema na requisição.")

    return results


def atualizar_csv():
    df_novo = pd.json_normalize(busca_tema()['articles'])

    try:
        df_antigo = pd.read_csv('code.csv', index_col=0)
        df_atualizado = pd.concat([df_antigo, df_novo]).drop_duplicates(subset=['publishedAt', 'author', 'title'], keep='last')
        df_atualizado.sort_values('publishedAt').reset_index(drop=True).to_csv('code.csv', index=True)
        print(f"Arquivo CSV atualizado com sucesso em {datetime.now()}.")

    except FileNotFoundError:
        df_novo.sort_values('publishedAt').reset_index(drop=True).to_csv('code.csv', index=True)
        print(f"Arquivo CSV criado com sucesso em {datetime.now()}.")

atualizar_csv()

<h3> Script dos relatórios (consultas) </h3>

<p> Para atender as demandas e possibilitar responder as perguntas dos usuários, os dados foram manipulados com pandas, permitindo a geraçao de relatórios diversos,
os quais foram armazenados em arquivos .csv </p>

O fluxograma para atingir o objetivo foi:
1. consultar a base dos dados;
2. manipular através da importação com pandas e utilizando o método 'groupby';
3. armazenar  através de '.csv' para disponibilizar posteriormente na API;

In [ ]:
import re
import pandas as pd
import requests
from datetime import datetime

df_historico = pd.read_csv("code.csv", index_col = 0)



# trasformacao;
df_historico['data'] = df_historico.publishedAt.apply(lambda x: x[:10])


# 4.1 - Quantidade de notícias por ano, mês e dia de publicação;
noticias_por_data = df_historico.groupby(['data'], as_index=False).count()[['data', 'title']]
noticias_por_data.columns = ['data', 'n_artigos']
noticias_por_data


#     - 4.2 - Quantidade de notícias por fonte e autor;
noticias_por_fonte_autor = df_historico.groupby(['source.name', 'author'], as_index=False).size()#.reset_index(name='Quantidade')
noticias_por_fonte_autor


# 4.3 - Quantidade de aparições das 3 palavras-chave por ano, mês e dia de publicação definidas no item 2
palavras_chave = ['bitcoin', 'blockchain', 'web3']
for palavra in palavras_chave:
    df_historico[palavra] = df_historico['title'].str.count(palavra, flags=re.IGNORECASE)

aparicoes_palavras_chave = df_historico.groupby(['data'], as_index = False)[palavras_chave].sum()
aparicoes_palavras_chave


# Salvando as transformações em arquivo csv
noticias_por_data.to_csv("noticias_por_data.csv",sep=",", index = False)
noticias_por_fonte_autor.to_csv("noticias_por_fonte_autor.csv",sep=",",index=False)
aparicoes_palavras_chave.to_csv("aparicoes_palavras_chave.csv",sep=",",index=False)


<p> Para permitir a consulta da base criada, foi utilizado a biblitoeca Flask, que permite inicializar uma REST API simples e funcional.</p>
<p> Os diferentes endpoints da API direcionam para os arquivos salvos no projeto, permitindo a leitura dos dados gerais, dados por ID
(através do método 'iloc' do pandas) e os diferentes relatórios no escopo do projeto ('flask_api.py'). </p>
<p> A API permite que o usuário submeta um artigo caso deseje e que não exista na base de dados, utilizando 'POST' ('post.py'). </p>

In [ ]:
# flask_api.py

import pandas as pd
import requests
#import pyspark.pandas as ps 
from flask import Flask, jsonify, request, make_response


# app.config["JSON_SORT_KEYS"] = False

app = Flask(
    __name__
)  # instância o método Flask com o nome do app igual ao nome do arquivo.py



dados = pd.read_csv('./code.csv')
noticias_por_data = pd.read_csv('./noticias_por_data.csv')
noticias_por_fonte = pd.read_csv('./noticias_por_fonte_autor.csv')
aparicoes_keyWords = pd.read_csv('./aparicoes_palavras_chave.csv')

@app.route("/", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def homepage():

    return jsonify({"mensagem": "olá, você está na página principal", "pagina": 1})

@app.route("/dados", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def todos_dados():

    
    return jsonify(
        dados.to_json())

@app.route("/dados/<int:id>", methods = ["GET"])
def retorna_id(id):

    try:
        df_id = dados.loc[[id]].to_json()
        return jsonify(
            df_id)
    except:
        print("id não encontrado")
        return jsonify(
            dados.to_json())

@app.route("/dados/noticias_data", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def noticias_dada():

    
    return jsonify(
        noticias_por_data.to_json())

@app.route("/dados/noticias_fonte", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def noticias_fonte():

    
    return jsonify(
        noticias_por_fonte.to_json())

@app.route("/dados/aparicoes_palavraschaves", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def aparicoes_chaves():

    
    return jsonify(
        aparicoes_keyWords.to_json())


@app.route("/dados", methods = ["POST"])
def recebe_noticia():

    novo_artigo = request.get_json()
    df_novoArtigo = pd.DataFrame(novo_artigo, index = [0])

    df = pd.read_csv('code.csv', index_col=0)

    df = pd.concat([df, df_novoArtigo]
            ).reset_index(drop = True)
    

    if df.duplicated(subset=['title', 'author']).sum() >=0:
        print("Artigo ja existente no banco de dados!")

        return jsonify(novo_artigo)

    # export.
    else:
        df.to_csv('code.csv')

        return jsonify(df.to_json())

##

if __name__ == "__main__":
    app.run(port = 5001, debug=True)

In [ ]:
# post.py

import requests
import pandas as pd

#inp == dados inputados pelo usuario para o POST;
inp = {'author': 'Test1', 'title': 'Test', 'description': 'more Test',
       'url': 'www.Test.com.br', 'urlToImage': 'htpps://Test', 
       'publishedAt': '2024-04-07 T19:00:00Z'}

url = 'http://127.0.0.1:5001/dados'

requests.post(url, json = inp)

In [ ]:
# schedule with CRON?

<h2> Resultados </h2>


In [ ]:
## colar uns prints aqui sobre os resultados obtidos :)